In [3]:
import cv2
import numpy as np
from PIL import Image
import os
import random
import json
import ast
import csv
from openpyxl import Workbook

## 4. Create new maps, with not-absolute location

In [4]:
def overlay_image(background, overlay, x, y, scale_factor):
    """Osadza overlay na background w pozycji (x, y) z obsługą alfa."""
    
    
    
    new_w = int(overlay.shape[1] * scale_factor)
    new_h = int(overlay.shape[0] * scale_factor)

    # Użyj cv2.resize
    overlay = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_LINEAR)


    bh, bw = background.shape[:2]
    oh, ow = overlay.shape[:2]

    if x + ow > bw or y + oh > bh:
        print("Ostrzeżenie: overlay wykracza poza obraz tła!")
       # print(x+ow, y+oh)
        return background

    # Wyodrębnij kanały
    b, g, r, a = cv2.split(overlay)
    mask = a / 255.0

    for c in range(3):  # RGB
        background[y:y+oh, x:x+ow, c] = (1.0 - mask) * background[y:y+oh, x:x+ow, c] + mask * overlay[:, :, c]

    # Nadpisujemy kanał alfa tła, jeśli potrzebne
    if background.shape[2] == 4:
        background[y:y+oh, x:x+ow, 3] = np.maximum(background[y:y+oh, x:x+ow, 3], a)

    return background

In [5]:
def load_json_file(json_path: str):
    """
    Loads a JSON file and returns its contents.

    Args:
        json_path (str): Path to the JSON file.

    Returns:
        object: Parsed JSON data (e.g., list of dicts).
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"✅ Wczytano {len(data)} areas z {json_path}")
    return data

In [6]:
def random_coord_in_area_with_size(area):
    """
    Losuje współrzędne w area tak, żeby cały gracz się zmieścił.
    """

    x = random.randint(area["x_min"], area["x_max"])
    y = random.randint(area["y_min"], area["y_max"])
    return (x, y)

In [7]:
def create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc):
    """
    Tworzy obraz postaci z nałożonymi elementami i zwraca go jako numpy array (RGBA).
    
    Args:
        hat_path (str): Ścieżka do pliku czapki.
        mask_path (str): Ścieżka do pliku maski/okularów.
        suit_path (str): Ścieżka do pliku kombinezonu.
        pc_path (str): Ścieżka do bazowej postaci.
    
    Returns:
        np.ndarray: Obraz wynikowy (z kanałem alfa).
    """
    r_pc = random.choice(sprites_pc)
    r_hat = random.choice(sprites_hats)
    r_eye = random.choice(sprites_eyes)
    r_pant = random.choice(sprites_pants)
    
    save_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total"

    # Plik główny (sprite)
    pc_path = "{}\\PC\\{}".format(save_path,  r_pc)
    
    # Dodatkowe elementy
    hat_path = "{}\\Hats\\{}".format(save_path, r_hat)
    mask_path = "{}\\Eyes\\{}".format(save_path,r_eye)
    suit_path = "{}\\Pants\\{}".format(save_path,r_pant)
    # Wczytaj obraz bazowy
   # print(pc_path)
    base_img = cv2.imread(pc_path, cv2.IMREAD_UNCHANGED)
    h, w = base_img.shape[:2]
    
    # Nowe wymiary
    new_w = w + 70 + 70
    new_h = h + 160 + 40

    # Tworzymy nowy obraz (przezroczysty)
    new_img = np.zeros((new_h, new_w, 4), dtype=np.uint8)

    # Wklejamy bazowy obraz na nowe płótno
    new_img[160:160+h, 50:50+w] = base_img

    # Wczytaj elementy
    hat = cv2.imread(hat_path, cv2.IMREAD_UNCHANGED)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    suit = cv2.imread(suit_path, cv2.IMREAD_UNCHANGED)

    # Osadź elementy
    extended_img = overlay_image(new_img, hat, 140, 60, scale_factor=1.5)
    extended_img = overlay_image(extended_img, mask, 200, 180, scale_factor=1.3)
    extended_img = overlay_image(extended_img, suit, 98, 355, scale_factor=2.1)

    return extended_img

In [8]:
def spawn_players_on_map_instant(
        sprites_hats, 
        sprites_eyes, 
        sprites_pants,
        sprites_pc,
        map_img,
        spawn_areas,
        output_image_path,
        num_players=1,
        char_width=100,
        char_height=150,
        scale = 4.4609375
                        ):
    """
    Wstawia graczy na mapę w losowych area.
    Zapisuje nowy obraz oraz plik JSON z koordynatami.
    """

    result_img = map_img.copy()

    # Zmień rozmiar postaci
  

    coords_list = []

        
    area = random.choice(spawn_areas)
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)
        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            # Kanał alfa
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

            
    # Koordynaty gracza
    player_coords = {'x': x, 'y': y}

    # Rozdzielczość monitora
    screen_size = (2560, 1440)

    img = result_img
    img_h, img_w = img.shape[:2]
    screen_w, screen_h = screen_size
    center_x, center_y = player_coords['x'], player_coords['y']

    # Oblicz granice cropa
    left = max(center_x - screen_w // 2, 0)
    top = max(center_y - screen_h // 2, 0)
    right = left + screen_w
    bottom = top + screen_h

    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)

    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    # Przytnij
    cropped = img[top:bottom, left:right]

    # Zapisz
    cv2.imwrite(output_image_path, cropped)
  #  print(coords_list)

    return coords_list



In [9]:
def save_coords_to_xlsx(data_list, xlsx_path):
    """
    Zapisuje listę (map_name, list of points) do pliku XLSX.

    Args:
        data_list (list): Lista krotek (map_name, [{'x': x, 'y': y}, ...])
        xlsx_path (str): Ścieżka do pliku XLSX.

    Returns:
        None
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "PlayerCoords"

    # Nagłówki
    ws.append(['map_name', 'point_index', 'x', 'y'])

    for map_name, points in data_list:
        for idx, point in enumerate(points):
            ws.append([map_name, idx, point['x'], point['y']])

    wb.save(xlsx_path)
    print(f"✅ Zapisano dane do {xlsx_path}")


In [10]:
import cv2
import random
import os

def spawn_players_on_map_instant_2(
        sprites_hats, 
        sprites_eyes, 
        sprites_pants,
        sprites_pc,
        map_img,
        spawn_areas,
        output_image_path,
        label_output_path,
        num_players,
        char_width=100,
        char_height=150,
        scale=4.4609375
    ):
    """
    Wstawia graczy w losowych miejscach na mapie,
    zapisuje screen o stałym rozmiarze 2560x1440 (losowy obszar mapy),
    generuje plik YOLO label .txt dla screena.
    """

    result_img = map_img.copy()
    coords_list = []

    # Losuj obszar do spawnu
    area = random.choice(spawn_areas)

    # Wstaw graczy na mapę
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)

        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

    # ===== Losowy crop screena =====
    screen_w, screen_h = 2560, 1440
    img_h, img_w = result_img.shape[:2]
    
    if num_players == 0:
        # losowy crop jak teraz
        max_left = max(img_w - screen_w, 0)
        max_top = max(img_h - screen_h, 0)
        left = random.randint(0, max_left)
        top = random.randint(0, max_top)
    else:
        # Dopasuj crop do graczy
        min_x = min(coord['x'] for coord in coords_list)
        max_x = max(coord['x'] for coord in coords_list)
        min_y = min(coord['y'] for coord in coords_list)
        max_y = max(coord['y'] for coord in coords_list)
    
        # wycentruj na środku graczy
        center_x = (min_x + max_x) // 2
        center_y = (min_y + max_y) // 2
    
        left = max(center_x - screen_w // 2, 0)
        top = max(center_y - screen_h // 2, 0)
    
    # korekta, żeby nie wyjść poza mapę
    right = left + screen_w
    bottom = top + screen_h
    
    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)
    
    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    cropped = result_img[top:bottom, left:right]

    # === Przelicz współrzędne względem screena ===
    screen_coords = []
    labels = []
    for coord in coords_list:
        screen_x = coord['x'] - left
        screen_y = coord['y'] - top

        if 0 <= screen_x <= screen_w and 0 <= screen_y <= screen_h:
            screen_coords.append({"x": screen_x, "y": screen_y})

            # YOLO format (normalized)
            x_center = (screen_x + char_width / 2) / screen_w
            y_center = (screen_y + char_height / 2) / screen_h
            w_norm = char_width / screen_w
            h_norm = char_height / screen_h

            labels.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # Zapisz screen
    cropped = apply_augmentations(cropped)
    cv2.imwrite(output_image_path, cropped)


    # Zapisz label
    with open(label_output_path, "w") as f:
        if labels:
            f.write("\n".join(labels))

    return screen_coords


In [11]:
import cv2
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter

def apply_augmentations(img, enable_flip=True):
    """Prosta augmentacja obrazu"""
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Jasność
    if random.random() < 0.5:
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(random.uniform(0.8, 1.2))

    # Kontrast
    if random.random() < 0.5:
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(random.uniform(0.8, 1.2))

    # Blur
    if random.random() < 0.1:
        img_pil = img_pil.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.5, 1.5)))

    # Flip poziomy
    if enable_flip and random.random() < 0.5:
        img_pil = img_pil.transpose(Image.FLIP_LEFT_RIGHT)

    # Nasycenie
    if random.random() < 0.5:
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(random.uniform(0.85, 1.15))

    img_aug = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img_aug

In [12]:
def spawn_players_on_map_instant_3(
        sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
        map_img, spawn_areas,
        output_image_path, output_label_path,
        num_players=1,
        char_width=100, char_height=150,
        scale=4.4609375,
        screen_size=(2560, 1440)
    ):
    """
    Generuje screen z graczami + YOLO label.
    """
    result_img = map_img.copy()
    coords_list = []

    # Spawn graczy
    area = random.choice(spawn_areas)
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = int(coord[0] * scale), int(coord[1] * scale)

        # Wklej postać
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)

        if char_img_resized.shape[2] == 4:
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

        coords_list.append({"x": x, "y": y})

    # === Kadrowanie ===
    screen_w, screen_h = screen_size
    img_h, img_w = result_img.shape[:2]

    if num_players == 0:
        # Losowy crop przy pustym screenie
        max_left = max(img_w - screen_w, 0)
        max_top = max(img_h - screen_h, 0)
        left = random.randint(0, max_left)
        top = random.randint(0, max_top)
    else:
        # Kadruj tak, żeby wszyscy gracze byli widoczni
        min_x = min(coord['x'] for coord in coords_list)
        max_x = max(coord['x'] for coord in coords_list)
        min_y = min(coord['y'] for coord in coords_list)
        max_y = max(coord['y'] for coord in coords_list)

        center_x = (min_x + max_x) // 2
        center_y = (min_y + max_y) // 2

        left = max(center_x - screen_w // 2, 0)
        top = max(center_y - screen_h // 2, 0)

    # Korekta kadru
    right = left + screen_w
    bottom = top + screen_h
    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)
    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    cropped = result_img[top:bottom, left:right]

    # === Augmentacja ===
    cropped = apply_augmentations(cropped)

    # === Generowanie YOLO label ===
    screen_coords = []
    label_lines = []
    for coord in coords_list:
        screen_x = coord['x'] - left
        screen_y = coord['y'] - top

        # Jeśli poza ekranem -> pomiń
        if screen_x < 0 or screen_y < 0 or screen_x > screen_w or screen_y > screen_h:
            continue

        # YOLO normalizacja
        x_center = (screen_x + char_width / 2) / screen_w
        y_center = (screen_y + char_height / 2) / screen_h
        w_norm = char_width / screen_w
        h_norm = char_height / screen_h

        # Clipping do [0,1]
        x_center = min(max(x_center, 0), 1)
        y_center = min(max(y_center, 0), 1)
        w_norm = min(max(w_norm, 0), 1)
        h_norm = min(max(h_norm, 0), 1)

        label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")
        screen_coords.append({"x": screen_x, "y": screen_y})

    # Zapisz obraz
    cv2.imwrite(output_image_path, cropped)

    # Zapisz YOLO labels
    with open(output_label_path, "w") as f:
        if label_lines:
            f.write("\n".join(label_lines))

    return screen_coords

In [13]:
areas = load_json_file("E:\\Project Sherlock\\spawn_areas.json")
# Ścieżki
map_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map.jpg"
map_img = cv2.imread(map_path)

sprites_pc = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\PC")[1:]
sprites_eyes = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Eyes")[1:]
sprites_pants = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")[1:]
sprites_hats = os.listdir("E:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats")[1:]

out_image_coordinates = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Coordinates_list_per_map.csv"
save_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total"

coordinates_list = []

✅ Wczytano 25 areas z E:\Project Sherlock\spawn_areas.json


In [14]:
spawn_areas=areas

In [15]:
import os
import random

# Foldery wyjściowe
output_images = "E:/Project Sherlock/Grafiki/Grafiki total/Generated_maps"
output_labels = "E:/Project Sherlock/Grafiki/Grafiki total/Generated_labels"
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

total_empty = 750
counts = {1: 250, 2: 250, 3: 250}

counter = 0

# === 1. Puste obrazy ===
for i in range(total_empty):
    file_name = f"Map_{counter}"
    out_image_path = os.path.join(output_images, file_name + ".jpg")
    out_label_path = os.path.join(output_labels, file_name + ".txt")

    coords = spawn_players_on_map_instant_2(
        sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
        map_img, spawn_areas,
        out_image_path, out_label_path,
        num_players=0  # brak graczy
    )

    print(f"[EMPTY] {file_name}.jpg | Players: {len(coords)}")
    counter += 1

# === 2. Obrazy z graczami ===
for num_players, target_count in counts.items():
    for j in range(target_count):
        file_name = f"Map_{counter}"
        out_image_path = os.path.join(output_images, file_name + ".jpg")
        out_label_path = os.path.join(output_labels, file_name + ".txt")

        coords = spawn_players_on_map_instant_3(
            sprites_hats, sprites_eyes, sprites_pants, sprites_pc,
            map_img, spawn_areas,
            out_image_path, out_label_path,
            num_players=num_players
        )

        print(f"[PLAYERS={num_players}] {file_name}.jpg | Players: {len(coords)}")
        counter += 1

print(f"✅ Wygenerowano {counter} plików (750 pustych + 750 z graczami)")


[EMPTY] Map_0.jpg | Players: 0
[EMPTY] Map_1.jpg | Players: 0
[EMPTY] Map_2.jpg | Players: 0
[EMPTY] Map_3.jpg | Players: 0
[EMPTY] Map_4.jpg | Players: 0
[EMPTY] Map_5.jpg | Players: 0
[EMPTY] Map_6.jpg | Players: 0
[EMPTY] Map_7.jpg | Players: 0
[EMPTY] Map_8.jpg | Players: 0
[EMPTY] Map_9.jpg | Players: 0
[EMPTY] Map_10.jpg | Players: 0
[EMPTY] Map_11.jpg | Players: 0
[EMPTY] Map_12.jpg | Players: 0
[EMPTY] Map_13.jpg | Players: 0
[EMPTY] Map_14.jpg | Players: 0
[EMPTY] Map_15.jpg | Players: 0
[EMPTY] Map_16.jpg | Players: 0
[EMPTY] Map_17.jpg | Players: 0
[EMPTY] Map_18.jpg | Players: 0
[EMPTY] Map_19.jpg | Players: 0
[EMPTY] Map_20.jpg | Players: 0
[EMPTY] Map_21.jpg | Players: 0
[EMPTY] Map_22.jpg | Players: 0
[EMPTY] Map_23.jpg | Players: 0
[EMPTY] Map_24.jpg | Players: 0
[EMPTY] Map_25.jpg | Players: 0
[EMPTY] Map_26.jpg | Players: 0
[EMPTY] Map_27.jpg | Players: 0
[EMPTY] Map_28.jpg | Players: 0
[EMPTY] Map_29.jpg | Players: 0
[EMPTY] Map_30.jpg | Players: 0
[EMPTY] Map_31.jpg